In [ ]:
%use lets-plot
%use plotly
import kotlinx.coroutines.*
import java.time.Duration
import java.time.Instant

val data = runBlocking { readAllUData("./log.anon") }

fun fillNoDataWith0(toFill: List<UData>): List<Pair<UInt, List<UData>>> {
    val filledQueryPerSecond = mutableListOf<Pair<UInt, List<UData>>>()
    var current = mutableListOf<UData>()
    var currentTime = toFill.first().unixTimeStamp
    toFill.forEach { u ->
        if (u.unixTimeStamp == currentTime) {
            current.add(u)
        } else {
            filledQueryPerSecond.add(Pair(currentTime, current))
            for (i in (currentTime + 1u)..(u.unixTimeStamp - 1u)) {
                // Empty list to add 0 values
                filledQueryPerSecond.add(Pair(i, listOf()))
            }
            current = mutableListOf(u)
            currentTime = u.unixTimeStamp
        }
    }
    filledQueryPerSecond.add(Pair(currentTime, current))
    return filledQueryPerSecond
}

val splitPerSuffix = data.groupBy { ud -> ud.domainName }.mapValues { (_, v) -> fillNoDataWith0(v) }

val queryPerSecond = fillNoDataWith0(data)

val amountOfSecondsWithoutValidations =
    queryPerSecond.last().first - queryPerSecond.first().first - (queryPerSecond.size.toUInt() - 1u)
if (amountOfSecondsWithoutValidations != 0u) {
    println("Error: $amountOfSecondsWithoutValidations seconds without validations.")
}

val times = queryPerSecond.map { t ->
    val time = t.first.toLong()
    Instant.ofEpochSecond(time).plus(Duration.ofHours(2)) // Convert to CET
}

fun plotOverTime(
    udata: Map<List<String>, List<Pair<UInt, List<UData>>>>,
    title: String,
    includeSub: Boolean = true
): Plot {

    val mergeSeconds = 60
    val merge = true

    fun merge(toMerge: List<Int>): List<Int> {
        return if (merge) {
            val mergedValidiations = mutableListOf<Int>()
            var i = 0
            var sum = 0
            toMerge.forEach { v ->
                sum += v
                i++
                if (i == mergeSeconds) {
                    mergedValidiations.add(sum)
                    sum = 0
                    i = 0
                }
            }
            if (sum != 0 && i != 0) {
                mergedValidiations.add(sum)
            }
            mergedValidiations
        } else {
            toMerge
        }
    }

    val timeString = if (merge) {
        var i = 0
        times.filter { _ ->
            val r = if (i == 0) {
                true
            } else {
                false
            }
            i = (i + 1) % mergeSeconds
            r
        }
    } else {
        times
    }.map { t -> t.toString() }

    val titleUpdated = if (merge) {
        "$title per $mergeSeconds seconds"
    } else {
        title
    }

    val yAxisText = if (merge) {
        if (mergeSeconds == 60) {
            "Amount of validations per minute"
        } else {
            "Amount of validations per $mergeSeconds seconds"
        }
    } else {
        "Amount of validations per second"
    }

    val perSuffixTraces = udata.map { (suffix, v) ->
        val secondsToSkip = v.first().first - data.first().unixTimeStamp
        val amountToSkip = if (!merge) {
            secondsToSkip.toInt()
        } else {
            secondsToSkip.toInt() / mergeSeconds
        }
        Trace() {
            name = suffix.joinToString(".")
            x.strings = timeString.drop(amountToSkip)
            y.numbers = merge(v.map { t -> t.second.filter { ud -> includeSub || ud.sub == false }.size })
            type = TraceType.scatter
        }
    }.sortedByDescending { t -> t.y.numbers.sumOf { n -> n.toInt() } }

    return Plotly.plot {
        traces(perSuffixTraces)
        scatter {
            line {
                width = 1
            }
        }

        layout {
            width = 1700
            height = 950
            title {
                text = "$titleUpdated"
            }
            xaxis {
                title {
                    text = "Time"
                }
            }
            yaxis {
                title {
                    text = yAxisText
                }
            }
        }
    }
}


In [ ]:
import java.time.OffsetDateTime
import java.time.ZoneOffset
import kotlin.system.exitProcess

val amount = queryPerSecond.map { t -> t.second.size }

val plotTime = mapOf<String, List<Any>>(
    "time" to times,
    "values" to amount
)

val plot = ggplot(plotTime) +
        geomLine { x = "time"; y = "values" } +
        ggtitle("Amount of validations over time") +
        ggsize(4000, 500)
plot.show()

In [ ]:
plotOverTime(mapOf(Pair(listOf("all"), queryPerSecond)), "Amount of validations over time")


In [ ]:
val amountRootZone =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size == 1 }) }

plotOverTime(mapOf(Pair(listOf(), amountRootZone)), "Amount of validations over time for root zone")

In [ ]:
val amountTLDs =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size <= 2 }) }

plotOverTime(
    mapOf(Pair(listOf("TLDs and root zone"), amountTLDs)),
    "Amount of validations over time for TLDs and root zone",
    false
)

In [ ]:
plotOverTime(splitPerSuffix, "Amount of validations over time per suffix including subdomains")

In [ ]:
plotOverTime(
    splitPerSuffix,
    "Amount of validations over time per suffix excluding subdomains",
    false
)